## Load Spot quadruped with arm

In [1]:
# load model
import gymnasium
import numpy as np
from IPython import display
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo

env = gymnasium.make(
    "Ant-v5",
    xml_file="./robots/boston_dynamics_spot/scene_arm.xml",
    forward_reward_weight=1,  # kept the same as the 'Ant' environment
    ctrl_cost_weight=0.05,  # changed because of the stronger motors of `Go1`
    contact_cost_weight=5e-4,  # kept the same as the 'Ant' environment
    healthy_reward=1,  # kept the same as the 'Ant' environment
    main_body=1,  # represents the "trunk" of the `Go1` robot
    healthy_z_range=(0.195, 0.95),
    include_cfrc_ext_in_observation=True,
    exclude_current_positions_from_observation=False,
    reset_noise_scale=0.1,
    frame_skip=25,
    max_episode_steps=1000,
    render_mode="rgb_array"
)

env = RecordVideo(env, video_folder="./videos", episode_trigger=lambda episode_id: True)
env = RecordEpisodeStatistics(env)

/home/ubuntu/final/Deep-Learning/DL/lib/python3.12/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/ubuntu/final/Deep-Learning/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [2]:
import os 
os.environ["MUJOCO_GL"] = "egl"

In [ ]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo
import mujoco

class LocoManipulationEnv(gym.Env):
    metadata = {
        "render_modes": ["human", "rgb_array"],
        "render_fps": 30
    }

    def __init__(self, render_mode=None, reward_fn=None):
        super().__init__()
        self.render_mode = render_mode
        self.reward_fn = reward_fn if reward_fn is not None else self.default_reward

        # Load MuJoCo model
        self.model = mujoco.MjModel.from_xml_path("./robots/boston_dynamics_spot/scene_arm.xml")
        self.data = mujoco.MjData(self.model)

        # Renderer for rgb_array mode
        if render_mode == "rgb_array":
            self.frame_width = 640
            self.frame_height = 480
            self._renderer = mujoco.Renderer(self.model, width=self.frame_width, height=self.frame_height)

        self.nq = self.model.nq
        self.nv = self.model.nv
        self.nu = self.model.nu

        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(self.nq + self.nv,),
            dtype=np.float32
        )

        self.action_space = spaces.Box(
            low=self.model.actuator_ctrlrange[:, 0],
            high=self.model.actuator_ctrlrange[:, 1],
            dtype=np.float32
        )

    def default_reward(self, obs, action):
        return -np.linalg.norm(obs)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        mujoco.mj_resetData(self.model, self.data)

        obs = np.concatenate([self.data.qpos, self.data.qvel]).astype(np.float32)
        return obs, {}

    def step(self, action):
        self.data.ctrl[:] = action
        mujoco.mj_step(self.model, self.data)

        obs = np.concatenate([self.data.qpos, self.data.qvel]).astype(np.float32)
        reward = self.reward_fn(obs, action)
        terminated = False
        truncated = False
        return obs, reward, terminated, truncated, {}

    def render(self):
        if self.render_mode == "rgb_array":
            self._renderer.update_scene(self.data)
            return self._renderer.render()
        elif self.render_mode == "human":
            if not hasattr(self, "viewer"):
                import mujoco.viewer
                self.viewer = mujoco.viewer.launch_passive(self.model, self.data)
            return None

    def close(self):
        if hasattr(self, "_renderer"):
            self._renderer.close()
        if hasattr(self, "viewer"):
            self.viewer.close()


In [4]:

reward_code = """
def reward_fn(obs, action):
    # Example: move end-effector to a goal with smooth control
    pos = obs[:3]  # Replace with correct index for end-effector
    goal = np.array([0.3, 0.2, 0.1])
    dist = np.linalg.norm(pos - goal)
    energy = np.linalg.norm(action)
    return -dist - 0.1 * energy
"""

local_vars = {}
exec(reward_code, {"np": np}, local_vars)
generated_reward_fn = local_vars["reward_fn"]


In [5]:
env = LocoManipulationEnv(render_mode="rgb_array", reward_fn=generated_reward_fn)
env = RecordVideo(env, video_folder="./videos", episode_trigger=lambda episode_id: True)
env = RecordEpisodeStatistics(env)


NameError: name 'mujoco' is not defined

## Episode loop

In [ ]:
for ep in range(4):
    print(f"Starting episode {ep + 1}")
    obs, _ = env.reset()
    done = False
    reward_total = 0
    step_count = 0

    while not done and step_count < 200:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, _ = env.step(action)
        env.render()
        reward_total += reward
        done = terminated or truncated
        step_count += 1

    print(f"Episode {ep + 1} ended with reward {reward_total:.2f}")

In [ ]:
import torch as t
import torch.nn as nn

class Actor(nn.Module):
    def __init__(self, obs_shape, action_shape, hidden_dim, std=0.1):
        super().__init__()

        self.std = std
        self.policy = nn.Sequential(nn.Linear(obs_shape[0], hidden_dim),
                                    nn.ReLU(inplace=True),
                                    nn.Linear(hidden_dim, hidden_dim),
                                    nn.ReLU(inplace=True),
                                    nn.Linear(hidden_dim, action_shape[0]))

        self.apply(utils.weight_init)

    def forward(self, obs):
        mu = self.policy(obs)
        mu = torch.tanh(mu)
        std = torch.ones_like(mu) * self.std

        dist = utils.TruncatedNormal(mu, std)
        return dist


In [ ]:
np.concatenate([[1,2], [3,4]])